In [206]:
import tensorflow as tf
import numpy as np

In [207]:
tf.config.experimental_run_functions_eagerly(True)

In [208]:
def generate_test_input(N):
    # Formato: (x1, y1, w1, h1, theta1)
    # Gerar boxes preditas e alvo aleatórias
    boxes_pred = np.random.rand(N, 5)*100  # N amostras de boxes preditas
   
    target_boxes = np.random.rand(N, 5)*100  # N amostras de boxes alvo

    # Adicionar valores específicos para o ângulo em radianos
    angle_values = np.random.uniform(low=0, high=2*np.pi, size=(N, 1))  # N valores de ângulo em radianos
    boxes_pred[:, 4] = angle_values.flatten()  # Substituição - Adicionando ângulos às boxes preditas
    target_boxes[:, 4] = angle_values.flatten()  # Substituição - Adicionando ângulos às boxes alvo
    
    return boxes_pred, target_boxes

In [209]:
boxes_predd, target_boxess = generate_test_input(1)

In [217]:
def probiou_loss_og(boxes_pred, target_boxes_, EPS = 1e-3, mode='l2'):

    """
        boxes_pred    -> a matrix [N,5](x,y,w,h,angle - in radians) containing ours predicted box ;in case of HBB angle == 0
        target_boxes_  -> a matrix [N,5](x,y,w,h,angle - in radians) containing ours target    box ;in case of HBB angle == 0
        EPS     -> threshold to avoid infinite values
        mode       -> ('l1' in [0,1] or 'l2' in [0,inf]) metrics according our paper

    """

    x1, y1, w1, h1, theta1 = tf.unstack(boxes_pred, axis=1)
    x2, y2, w2, h2, theta2 = tf.unstack(target_boxes_, axis=1)
    x1 = tf.reshape(x1, [-1, 1])
    y1 = tf.reshape(y1, [-1, 1])
    h1 = tf.reshape(h1, [-1, 1])
    w1 = tf.reshape(w1, [-1, 1])
    theta1 = tf.reshape(theta1, [-1, 1])
    x2 = tf.reshape(x2, [-1, 1])
    y2 = tf.reshape(y2, [-1, 1])
    h2 = tf.reshape(h2, [-1, 1])
    w2 = tf.reshape(w2, [-1, 1])
    theta2 = tf.reshape(theta2, [-1, 1])

    # gbb form
    aa = w1**2/12; bb = h1**2/12; angles = theta1
    # rotated form
    a1 = aa*tf.math.pow(tf.math.cos(angles), 2.) + bb*tf.math.pow(tf.math.sin(angles), 2.)
    b1 = aa*tf.math.pow(tf.math.sin(angles), 2.) + bb*tf.math.pow(tf.math.cos(angles), 2.)
    c1 = 0.5*(aa - bb)*tf.math.sin(2.*angles)

    # gbb form
    aa = w2**2/12; bb = h2**2/12; angles = theta2
    # rotated form
    a2 = aa*tf.math.pow(tf.math.cos(angles), 2.) + bb*tf.math.pow(tf.math.sin(angles), 2.)
    b2 = aa*tf.math.pow(tf.math.sin(angles), 2.) + bb*tf.math.pow(tf.math.cos(angles), 2.)
    c2 = 0.5*(aa - bb)*tf.math.sin(2.*angles)

    B1 = 1/4.*( (a1+a2)*(y1-y2)**2. + (b1+b2)*(x1-x2)**2. ) + 1/2.*( (c1+c2)*(x2-x1)*(y1-y2) )
    # B1 = B1 / ( (a1+a2)*(b1+b2) - (c1+c2)**2. + EPS )
    B1 = B1 / ( (a1+a2)*(b1+b2) - (c1+c2)**2.)

    print("valor de b1 na loss OG:{}".format(B1))
    sqrt = (a1*b1-c1**2)*(a2*b2-c2**2)
    # sqrt = tf.clip_by_value(sqrt, EPS, tf.reduce_max(sqrt)+EPS)
    # sqrt = tf.clip_by_value(sqrt, EPS, tf.reduce_max(sqrt))
    # B2 = ( (a1+a2)*(b1+b2) - (c1+c2)**2. )/( 4.*tf.math.sqrt(sqrt) + EPS )
    B2 = ( (a1+a2)*(b1+b2) - (c1+c2)**2. )/( 4.*tf.math.sqrt(sqrt))
    #B2 = tf.clip_by_value(B2, EPS, tf.reduce_max(B2)+EPS)
    B2 = 1/2.*tf.math.log(B2)
    
    #print(B2.shape)

    #print("valor de b2 na LOSS OG{}".format(B2))
    
    Bd = B1 + B2
    Bd = tf.clip_by_value(Bd, EPS, 100.)
    
    print(Bd.shape)

    l1 = tf.math.sqrt(1 - tf.math.exp(-Bd) + EPS)

    if mode=='l2':
        l2 = tf.math.pow(l1, 2.)
        probiou = - tf.math.log(1. - l2 + EPS)
    else:
        probiou = l1

    return B1

In [255]:
def probiou_lossTF(boxes_pred, target_boxes_, EPS = 1e-3, mode='l2'):

    """
        boxes_pred    -> a matrix [N,5](x,y,w,h,angle - in radians) containing ours predicted box ;in case of HBB angle == 0
        target_boxes_  -> a matrix [N,5](x,y,w,h,angle - in radians) containing ours target    box ;in case of HBB angle == 0
        EPS     -> threshold to avoid infinite values
        mode       -> ('l1' in [0,1] or 'l2' in [0,inf]) metrics according our paper

    """

    x1, y1, w1, h1, theta1 = tf.unstack(boxes_pred, axis=1)
    x2, y2, w2, h2, theta2 = tf.unstack(target_boxes_, axis=1)
    x1 = tf.reshape(x1, [-1])
    y1 = tf.reshape(y1, [-1])
    h1 = tf.reshape(h1, [-1])
    w1 = tf.reshape(w1, [-1])
    theta1 = tf.reshape(theta1, [-1])
    x2 = tf.reshape(x1, [-1])
    y2 = tf.reshape(y2, [-1])
    h2 = tf.reshape(h2, [-1])
    w2 = tf.reshape(w2, [-1])
    theta2 = tf.reshape(theta2, [-1])

    # gbb form
    # print("O valor de w1 eh: {}".format(w1))
    # print("O valor de h1 eh: {}".format(h1))
    # print("O valor de theta1 eh: {}".format(theta1))
    aa = w1**2/12; bb = h1**2/12; angles = theta1
    # rotated form
    a1 = aa*tf.math.pow(tf.math.cos(angles), 2.) + bb*tf.math.pow(tf.math.sin(angles), 2.)
    b1 = aa*tf.math.pow(tf.math.sin(angles), 2.) + bb*tf.math.pow(tf.math.cos(angles), 2.)
    c1 = 0.5*(aa - bb)*tf.math.sin(2.*angles)

    # gbb form
    aa = w2**2/12; bb = h2**2/12; angles = theta2
    # rotated form
    a2 = aa*tf.math.pow(tf.math.cos(angles), 2.) + bb*tf.math.pow(tf.math.sin(angles), 2.)
    b2 = aa*tf.math.pow(tf.math.sin(angles), 2.) + bb*tf.math.pow(tf.math.cos(angles), 2.)
    c2 = 0.5*(aa - bb)*tf.math.sin(2.*angles)
    
    # print(tf.rank(a1))
    # print("a1:{}".format(a1))
    # print("c1:{}".format(c1))
    # print("b1:{}".format(b1))
    
    # Formulação matricial
    sigma1 = tf.transpose(tf.Variable([[a1, c1], [c1, b1]],), perm=[2, 1, 0]) # sigma1 tem dim (n, 2, 2)
    
    sigma2 = tf.transpose(tf.Variable([[a2, c2], [c2, b2]]), perm=[2, 1, 0])  # sigma2 tem dim (n, 2, 2)
    
    sigma = (1/2)*(sigma1 + sigma2)
    
    mu1 = tf.transpose(tf.expand_dims(tf.Variable([x1, y1]), axis=-1), perm=[1, 0, 2]) # mu1 tem dim (n, 2, 1)
    
    mu2 = tf.transpose(tf.expand_dims(tf.Variable([x2, y2]), axis=-1), perm=[1, 0, 2]) # mu2 tem dim (n, 2, 1)
    
    b1 = (1/8)*tf.linalg.matmul(tf.transpose((mu1 - mu2), perm=[0, 2, 1]), tf.linalg.inv(sigma))
    b1 = tf.squeeze(tf.linalg.matmul(b1, (mu1 - mu2)), axis=2)
    
    print(b1)
    
    b2 = (1/2)*tf.math.log(tf.linalg.det(sigma)/tf.math.sqrt(tf.linalg.det(sigma1)*tf.linalg.det(sigma2)))
    #print("valor de pt2: ", b2)
    
    Bd = b1+b2
    
    Bd = tf.clip_by_value(Bd, EPS, 100.)

    l1 = tf.math.sqrt(1 - tf.math.exp(-Bd) + EPS)
    
    if mode=='l2':
        l2 = tf.math.pow(l1, 2.)
        probiou = - tf.math.log(1. - l2 + EPS)
    else:
        probiou = l1
        
    return b1


In [256]:
digs3 = probiou_lossTF(boxes_predd, target_boxess)

 valor de sigma:  tf.Tensor(
[[[1.5 2.5]
  [2.5 5. ]]], shape=(1, 2, 2), dtype=float64)
 valor de sigmainv:  tf.Tensor(
[[[ 4.  -2. ]
  [-2.   1.2]]], shape=(1, 2, 2), dtype=float64)
 valor de testedebug2:  tf.Tensor([[[0.6]]], shape=(1, 1, 1), dtype=float64)
tf.Tensor(
[[[2.]
  [2.]]], shape=(1, 2, 1), dtype=float64)
tf.Tensor([[0.6]], shape=(1, 1), dtype=float64)


In [220]:
digs2 = probiou_loss_og(boxes_predd, target_boxess)

valor de b1 na loss OG:[[1.13408398]]
(1, 1)


In [258]:
a1=tf.Variable([1], dtype=tf.float64) # elementos matriz sigma1
c1=tf.Variable([2], dtype=tf.float64) # 1 2
b1=tf.Variable([6], dtype=tf.float64) # 2 6
    
a2=tf.Variable([2], dtype=tf.float64) # elementos matriz sigma2
c2=tf.Variable([3], dtype=tf.float64) # 2 3
b2=tf.Variable([4], dtype=tf.float64) # 3 4
    
x1=tf.Variable([4], dtype=tf.float64) # 4 elementos matriz mu1
y1=tf.Variable([4], dtype=tf.float64) # 4
    
x2=tf.Variable([2], dtype=tf.float64) # 2 elementos matriz mu2 
y2=tf.Variable([2], dtype=tf.float64) # 2

sigma1 = tf.transpose(tf.Variable([[a1, c1], [c1, b1]],), perm=[2, 1, 0]) # sigma1 tem dim (n, 2, 2)
    
sigma2 = tf.transpose(tf.Variable([[a2, c2], [c2, b2]]), perm=[2, 1, 0])  # sigma2 tem dim (n, 2, 2)

mu1 = tf.transpose(tf.expand_dims(tf.Variable([x1, y1]), axis=-1), perm=[1, 0, 2]) # mu1 tem dim (n, 2, 1)
    
mu2 = tf.transpose(tf.expand_dims(tf.Variable([x2, y2]), axis=-1), perm=[1, 0, 2]) # mu2 tem dim (n, 2, 1)
    
sigma = (1/2)*(sigma1 + sigma2)
    
sigmainv = tf.linalg.inv(sigma)
    
print(" valor de sigma: ", sigma)
    
print(" valor de sigmainv: ", sigmainv)

testedebug = (1/8)*tf.linalg.matmul(tf.transpose((mu1-mu2), perm=[0, 2, 1]), tf.linalg.inv(sigma))
    
testedebug2 = tf.linalg.matmul(testedebug, (mu1 - mu2))
    
print(" valor de testedebug2: ", testedebug2)

 valor de sigma:  tf.Tensor(
[[[1.5 2.5]
  [2.5 5. ]]], shape=(1, 2, 2), dtype=float64)
 valor de sigmainv:  tf.Tensor(
[[[ 4.  -2. ]
  [-2.   1.2]]], shape=(1, 2, 2), dtype=float64)
 valor de testedebug2:  tf.Tensor([[[0.6]]], shape=(1, 1, 1), dtype=float64)
